In [0]:
#2.Using spark sql Transformations such as Filter, Join, Simple Aggregations, GroupBy on the case study dataset

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("CreditCardAnalysis").getOrCreate()

# Load dataset
credit_card_df = spark.read.csv("/FileStore/tables/credit_card-1.csv", header=True, inferSchema=True)
new_credit=spark.read.csv("/FileStore/tables/book.csv", header=True, inferSchema=True)

# Create a temporary SQL view
credit_card_df.createOrReplaceTempView("credit_card")
new_credit.createOrReplaceTempView("new")


+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

In [0]:
# 1.Using FIlter

# 1. Filter transactions where credit score is above 830
spark.sql("SELECT * FROM credit_card WHERE CreditScore > 830").show()




+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        5|  15737888|  Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|       25|  15625047|       Yen|        846|   France|Female| 38|     5|      0.0|            1|             1|      187616.16|     0|
|       39|  15717426| Armstrong|        850|   France|  Male| 36|     7|      0.0|            1|             1|        40812.9|     0|
|       44|  15755196|    Lavine|        834|   France|Female| 49|     2|131394.56|            1|             0|      194365.76|     1|
|      181|  15716334|    Rozier|        850|   

In [0]:
# 2. FIlter for female customer

spark.sql("SELECT * FROM credit_card WHERE Gender = 'Female'").show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|  Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|      Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|      Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|      Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|  Mitchell|        850|   

In [0]:
# 2.Using joins
# 1. Inner join on customer_id

spark.sql("""
    SELECT cc.*
    FROM credit_card cc
    INNER JOIN new n
    ON n.CustomerId = cc.CustomerId
""").show(5)


# 2. Outer join on customer_id
spark.sql("""
    SELECT cc.*
    FROM credit_card cc
    full OUTER JOIN new n
    ON n.CustomerId = cc.CustomerId
""").show(5)

# 3.Left join
spark.sql("""
    SELECT cc.*
    FROM credit_card cc
    left JOIN new n
    ON n.CustomerId = cc.CustomerId
""").show(5)

# Right join
spark.sql("""
    SELECT cc.*
    FROM credit_card cc
    Right JOIN new n
    ON n.CustomerId = cc.CustomerId
""").show(5)

#Left Anti join
spark.sql("""
    SELECT cc.*
    FROM credit_card cc
    LEFT ANTI JOIN new n
    ON n.CustomerId = cc.CustomerId
""").show(5)

#left semi join
spark.sql("""
    SELECT cc.*
    FROM credit_card cc
    LEFT SEMI JOIN new n
    ON n.CustomerId = cc.CustomerId
""").show(5)



+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

In [0]:
# Using Simple Aggregations
# 1. Total Balance Amount
spark.sql("SELECT SUM(Balance) AS total_amount FROM credit_card").show()

#2.Average of Balance amount
spark.sql("SELECT avg(Balance) AS Average FROM credit_card").show()


+-------------------+
|       total_amount|
+-------------------+
|7.648588928799961E8|
+-------------------+

+-----------------+
|          Average|
+-----------------+
|76485.88928799961|
+-----------------+



In [0]:
# Using group by
# 2. Average transaction amount by region
spark.sql("""
    SELECT Geography, AVG(Balance) AS avg_spent
    FROM credit_card
    GROUP BY Geography
""").show()

+---------+------------------+
|Geography|         avg_spent|
+---------+------------------+
|  Germany|119730.11613391782|
|   France|  62092.6365157559|
|    Spain| 61818.14776342349|
+---------+------------------+



**Pyspark Implementation**


In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Credit Card Analysis") \
    .getOrCreate()

# Load the dataset
file_path = '/FileStore/tables/credit_card-1.csv'  
credit_df = spark.read.csv(file_path, header=True, inferSchema=True)

filenew = '/FileStore/tables/book.csv'
new_df=spark.read.csv(filenew, header=True, inferSchema=True)

In [0]:
# Filter transactions where CreditScore > 830 using DataFrame API
filtered_df = credit_df.filter(credit_df["CreditScore"] > 830)

# Show the results
filtered_df.show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        5|  15737888|  Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|       25|  15625047|       Yen|        846|   France|Female| 38|     5|      0.0|            1|             1|      187616.16|     0|
|       39|  15717426| Armstrong|        850|   France|  Male| 36|     7|      0.0|            1|             1|        40812.9|     0|
|       44|  15755196|    Lavine|        834|   France|Female| 49|     2|131394.56|            1|             0|      194365.76|     1|
|      181|  15716334|    Rozier|        850|   

In [0]:
# Filter for female customers using DataFrame API
female_customers_df = credit_df.filter(credit_df["Gender"] == "Female")

# Show the results
female_customers_df.show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|  Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|      Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|      Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|      Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|  Mitchell|        850|   

In [0]:
from pyspark.sql.functions import sum, avg
# 1. Total Balance Amount
total_balance = credit_df.agg(sum("Balance").alias("TotalBalance"))
total_balance.show()

# 2. Average of Balance Amount
average_balance = credit_df.agg(avg("Balance").alias("AverageBalance"))
average_balance.show()

+-------------------+
|       TotalBalance|
+-------------------+
|7.648588928799961E8|
+-------------------+

+-----------------+
|   AverageBalance|
+-----------------+
|76485.88928799961|
+-----------------+



In [0]:
# Group by Geography and calculate the average transaction amount using DataFrame API
avg_transaction_by_region = credit_df.groupBy("Geography").avg("Balance").alias("avg_spent"))

# Show the results
avg_transaction_by_region.show()

+---------+------------------+
|Geography|      avg(Balance)|
+---------+------------------+
|  Germany|119730.11613391782|
|   France|  62092.6365157559|
|    Spain| 61818.14776342349|
+---------+------------------+



In [0]:
#Joins
joined_df = credit_df.join(new_df, credit_df["CustomerId"] == new_df["CustomerId"], "inner").show()
joined_df = credit_df.join(new_df, credit_df["CustomerId"] == new_df["CustomerId"], "outer").show()
joined_df = credit_df.join(new_df, credit_df["CustomerId"] == new_df["CustomerId"], "left").show()
joined_df = credit_df.join(new_df, credit_df["CustomerId"] == new_df["CustomerId"], "right").show()
joined_df = credit_df.join(new_df, credit_df["CustomerId"] == new_df["CustomerId"], "leftanti").show()
joined_df = credit_df.join(new_df, credit_df["CustomerId"] == new_df["CustomerId"], "leftsemi").show()


+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|        1|  15634602|Hargrave|        619|   France|Female| 42|     2